In [1]:
# type:ignore
import os
import time
import pyodbc
import pandas as pd

In [2]:
# 建立连接并列出表
def get_tables(conn_str):
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    try:
        # 获取所有表名
        tables = cursor.tables(tableType='TABLE')
        tables_name = [table.table_name for table in tables]
        return tables_name
    finally:
        cursor.close()
        conn.close()

# 读取表数据，返回一个pandas的DataFrame
def get_table_data(conn_str,table_name):
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    try:
        cursor.execute(f"SELECT * FROM {table_name}")
        # 获取列名
        columns = [column[0] for column in cursor.description]
        # 获取数据并创建DataFrame,指定列名
        data = pd.DataFrame.from_records(cursor.fetchall(), columns=columns)
        return data
    finally:
        cursor.close()
        conn.close()
# 查看DF的某列是否有重复值
def check_duplicate(df,column_name):
    return df[column_name].duplicated().sum()

# 去除DF中某列的重复值(只保留第一个)
def remove_duplicate(df,column_name):
    return df.drop_duplicates(subset=[column_name],keep='first')


In [3]:
# 指定MDB文件路径 - 需要包含文件扩展名
mdb_file = r"C:\Users\Runker\Desktop\wcx\mdb\520326.mdb"  # 添加.mdb扩展名

# 检查文件是否存在
if not os.path.exists(mdb_file):
    raise FileNotFoundError(f"找不到MDB文件: {mdb_file}")

# 使用ODBC连接字符串
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=' + mdb_file + ';'
)

In [4]:
# 获取所有表名
tables = get_tables(conn_str)
print(tables,len(tables))


['CYXX', 'DCYD', 'FJWJ', 'GDB_ColumnInfo', 'JCFF', 'LDTJDCXX', 'PMXTXDCFCXX', 'PMXTXDCJBXX', 'SFYL', 'SXDM', 'SYS', 'TRHJXZ', 'TRHXXZ', 'TRRZ', 'TRWLXZ', 'YDSQ', 'YPLZ', 'YPLZQD', 'YPZB', 'ZKYP'] 20


In [5]:
# 读取表数据 调查样点属性表  use_col YDBH YDLB CYLX BSJD BSWD JD WD DWJD WDWD DWGC TL YL TS TZ SFCJSWXDTJTYP SFJCJXZC SFTTCPYD TTCPMC
dcyd_df = get_table_data(conn_str,'DCYD')
# 读取指定列
dcyd_df = dcyd_df[['YDBH','YDLB','CYLX','BSJD','BSWD','JD','WD','DWJD','DWWD','DWGC',
                   'TL','YL','TS','TZ','SFCJSWXDTJTYP','SFJCJXZC','SFTTCPYD','TTCPMC']]
print(dcyd_df.shape[0])
# 查看唯一的YDBH
print(dcyd_df['YDBH'].nunique())
# 更改列名TS为TSS避免与其他表格冲突
dcyd_df.rename(columns={'TS':'TSS'},inplace=True)
dcyd_df.sample(1)

1802
1802


,YDBH,YDLB,CYLX,BSJD,BSWD,JD,WD,DWJD,DWWD,DWGC,TL,YL,TSS,TZ,SFCJSWXDTJTYP,SFJCJXZC,SFTTCPYD,TTCPMC
1482,5203260103001711,0,2,108.14651020859372000000,28.79035549084723700000,108.14651020859372000000,28.79035549084723700000,108.14622495999018000000,28.79053499743341500000,725.61251742765310000000,石灰（岩）土,黑色石灰土,黑色石灰壤土,薄层黑色石灰壤土,1,1,0,None


In [6]:
# 读取表数据 采样信息属性表 use_col YDBH YPBH YPLX CH
cyxx_df = get_table_data(conn_str,'CYXX')
# 读取指定列
cyxx_df = cyxx_df[['YDBH','YPBH','YPLX','CH']]
print(cyxx_df.shape[0])
# 查看唯一的YDBH 
print(cyxx_df['YDBH'].nunique())
cyxx_df.sample(1)

3867
1802


,YDBH,YPBH,YPLX,CH
2280,5203260103001515,520326010300151510,1,None


In [7]:
# 读取表数据 土壤物理性状属性表 use_col YDBH YPBH YYPBH JXZCXSL JXZC1 JXZC2 JXZC3 JXZC4 TRZD SWXDTJT1 SWXDTJT2 SWXDTJT3 SWXDTJT4 SWXDTJT5 SWXDTJT6 SWXDTJT7 
wlxz_df = get_table_data(conn_str,'TRWLXZ')
# 读取指定列
wlxz_df = wlxz_df[['YDBH','YPBH','YYPBH','JXZCXSL','JXZC1','JXZC2','JXZC3','JXZC4','TRZD','SWXDTJT1','SWXDTJT2','SWXDTJT3','SWXDTJT4','SWXDTJT5','SWXDTJT6','SWXDTJT7']]
print(wlxz_df.shape[0])
# 查看唯一的YDBH 
print(wlxz_df['YDBH'].nunique())
wlxz_df.sample(1)


2130
1802


,YDBH,YPBH,YYPBH,JXZCXSL,JXZC1,JXZC2,JXZC3,JXZC4,TRZD,SWXDTJT1,SWXDTJT2,SWXDTJT3,SWXDTJT4,SWXDTJT5,SWXDTJT6,SWXDTJT7
360,5203260101001136,240228151572b39,520326010100113610,None,None,None,None,None,None,None,None,None,None,None,None,None


In [8]:
# 读取表数据 土壤化学性状属性表 use_col YDBH YPBH YYPBH FGSYHSL PH JHXSZL ECH ECAL SJXZSD CEC JHXYJZL ECA EMG ENA EK SRXYZL DDL SRXNLZ SRXJLZ SRXGLZ
# SRXMLZ SRXTSG SRXTSQG SRXLSG SRXLG LZZL OM TN TP TK TS TB TSI TSE TFE TMN TCU TZN TMO TAL TCA TMG AP SK AK AS1 ASI AFE AMN ACU AZN AB AMO CACO3 FE2O3
hxxz_df = get_table_data(conn_str,'TRHXXZ')
# 读取指定列
hxxz_df = hxxz_df[['YDBH','YPBH','YYPBH','FGSYHSL','PH','JHXSZL','ECH','ECAL','SJXZSD','CEC','JHXYJZL','ECA','EMG','ENA',
                   'EK','SRXYZL','DDL','SRXNLZ','SRXJLZ','SRXGLZ','SRXMLZ','SRXTSG','SRXTSQG','SRXLSG','SRXLG','LZZL',
                   'OM','TN','TP','TK','TS','TB','TSI','TSE','TFE','TMN','TCU','TZN','TMO','TAL','TCA','TMG','AP',
                   'SK','AK','AS1','ASI','AFE','AMN','ACU','AZN','AB','AMO','CACO3','FE2O3']]
print(hxxz_df.shape[0])
# 查看唯一的YDBH 
print(hxxz_df['YDBH'].nunique())
hxxz_df.sample(1)

1929
1802


,YDBH,YPBH,YYPBH,FGSYHSL,PH,JHXSZL,ECH,ECAL,SJXZSD,CEC,...,AS1,ASI,AFE,AMN,ACU,AZN,AB,AMO,CACO3,FE2O3
1490,5203260103001098,240110132039b30,520326010300109810,3.9,5.52,None,None,None,None,11.1,...,15.1,None,70.0,55.7,4.10,0.90,0.21,0.634,None,None


In [9]:
# 读取表数据 土壤环境性状属性表 use_col YDBH YPBH YYPBH HG AS2 PB CD CR NI
hjxz_df = get_table_data(conn_str,'TRHJXZ')
# 读取指定列
hjxz_df = hjxz_df[['YDBH','YPBH','YYPBH','HG','AS2','PB','CD','CR','NI']]
print(hjxz_df.shape[0])
# 查看唯一的YDBH 
print(hjxz_df['YDBH'].nunique())
hjxz_df.sample(1)


1929
1802


,YDBH,YPBH,YYPBH,HG,AS2,PB,CD,CR,NI
214,5203260103100020,240429181742b14,520326010310002013,0.300,30.2,45.5,0.08,84.2,50.5


In [10]:
# 读取表数据 土壤容重属性表 use_col YDBH YPBH TRRZ1 TRRZ2 TRRZ3 TRRZ4 TRRZPJZ 
trrz_df = get_table_data(conn_str, 'TRRZ')
# 读取指定列
trrz_df = trrz_df[['YDBH','YPBH','TRRZ1','TRRZ2','TRRZ3','TRRZ4','TRRZPJZ']]
print(trrz_df.shape[0])
# 查看唯一的YDBH 
print(trrz_df['YDBH'].nunique())
trrz_df.sample(1)


1786
1712


,YDBH,YPBH,TRRZ1,TRRZ2,TRRZ3,TRRZ4,TRRZPJZ
781,5203260103000754,520326010300075420,1.40000000000000000000,1.35000000000000000000,1.26000000000000000000,None,1.34000000000000000000


In [11]:
# 读取表数据 立地条件调查表  use_col YDBH MY MYQT MZ MZQT TDLYLX GZCHD
ldtj_df = get_table_data(conn_str,'LDTJDCXX')
# 读取指定列
ldtj_df = ldtj_df[['YDBH','MY','MYQT','MZ','MZQT','TDLYLX','GZCHD']]
print(ldtj_df.shape[0])
# 查看唯一的YDBH 
print(ldtj_df['YDBH'].nunique())
ldtj_df.sample(1)

1802
1802


,YDBH,MY,MYQT,MZ,MZQT,TDLYLX,GZCHD
1695,5203260103000596,27,None,LI,None,0103,15.20000000000000000000


In [12]:
# 读取表数据 剖面形态学调查基本信息调查表  use_col YDBH FSCS YXTCHD FSXTL FSXYL TSXTS TSXTZ 
pmxx_df = get_table_data(conn_str,'PMXTXDCJBXX')
# 读取指定列
pmxx_df = pmxx_df[['YDBH','FSCS','YXTCHD','FSXTL','FSXYL','FSXTS','FSXTZ']]
print(pmxx_df.shape[0])
# 查看唯一的YDBH 
print(pmxx_df['YDBH'].nunique())
pmxx_df.sample(1)

29
29


,YDBH,FSCS,YXTCHD,FSXTL,FSXYL,FSXTS,FSXTZ
3,5203260305100027,3,78,石灰（岩）土,黄色石灰土,黏质黄色石灰土,腐中层黏质黄色石灰土


In [13]:
# 土壤容重结果 ，基于YDBH
trrz_result = pd.merge(dcyd_df,trrz_df,on='YDBH',how='inner')
print(trrz_result.shape[0])
trrz_result.sample(1)

1786


,YDBH,YDLB,CYLX,BSJD,BSWD,JD,WD,DWJD,DWWD,DWGC,...,SFCJSWXDTJTYP,SFJCJXZC,SFTTCPYD,TTCPMC,YPBH,TRRZ1,TRRZ2,TRRZ3,TRRZ4,TRRZPJZ
606,5203260103000502,0,2,107.87539146426037000000,28.68401016420756000000,107.87539146426037000000,28.68401016420756000000,107.87506666666668000000,28.68445333333333400000,641.30000000000000000000,...,0,0,0,None,520326010300050220,1.22000000000000000000,1.23000000000000000000,1.29000000000000000000,None,1.25000000000000000000


In [14]:
# 土壤物理性状结果 ，需要连接DCYD和CYXX，基于YDBH
temp_result = pd.merge(dcyd_df,cyxx_df,on='YDBH',how='inner')
print(temp_result.shape[0])
wlxz_result = pd.merge(temp_result,wlxz_df,left_on='YPBH',right_on='YYPBH',how='inner')
print(wlxz_result.shape[0])
wlxz_result.sample(1)

3867
2130


,YDBH_x,YDLB,CYLX,BSJD,BSWD,JD,WD,DWJD,DWWD,DWGC,...,JXZC3,JXZC4,TRZD,SWXDTJT1,SWXDTJT2,SWXDTJT3,SWXDTJT4,SWXDTJT5,SWXDTJT6,SWXDTJT7
182,5203260101001007,0,1,107.81119555256578000000,28.45010639241330400000,107.81119555256578000000,28.45010639241330400000,107.81164728855457000000,28.44999970780014200000,1002.12040024716410000000,...,33.9,25.6,壤质黏土,None,None,None,None,None,None,None


In [15]:
# 土壤化学性状结果 ，需要连接DCYD和CYXX，基于YDBH
temp_result = pd.merge(dcyd_df,cyxx_df,on='YDBH',how='inner')
print(temp_result.shape[0])
hxxz_result = pd.merge(temp_result,hxxz_df,left_on='YPBH',right_on='YYPBH',how='inner')
print(hxxz_result.shape[0])
hxxz_result.sample(1)


3867
1929


,YDBH_x,YDLB,CYLX,BSJD,BSWD,JD,WD,DWJD,DWWD,DWGC,...,AS1,ASI,AFE,AMN,ACU,AZN,AB,AMO,CACO3,FE2O3
32,5203260101000181,0,1,107.88782788168629000000,28.71570503871444700000,107.88782788168629000000,28.71570503871444700000,107.88813983000000000000,28.71576319000000000000,620.94055175781250000000,...,7.23,233,14.8,12.8,1.65,1.20,0.15,0.072,None,None


In [16]:
# 土壤环境性状结果 ，需要连接DCYD和CYXX，基于YDBH
temp_result = pd.merge(dcyd_df,cyxx_df,on='YDBH',how='inner')
print(temp_result.shape[0])
hjxz_result = pd.merge(temp_result,hjxz_df,left_on='YPBH',right_on='YYPBH',how='inner')
print(hjxz_result.shape[0])
hjxz_result.sample(1)


3867
1929


,YDBH_x,YDLB,CYLX,BSJD,BSWD,JD,WD,DWJD,DWWD,DWGC,...,CH,YDBH_y,YPBH_y,YYPBH,HG,AS2,PB,CD,CR,NI
1516,5203260103001647,0,1,108.07925098749148000000,28.88343230568438700000,108.07925098749148000000,28.88343230568438700000,108.07936950857886000000,28.88354139742116000000,1096.89559038355950000000,...,None,5203260103001647,240227151383b44,520326010300164710,0.155,15.7,30.5,0.39,62.6,24.0


In [17]:
def process_merge_result(df):
    """
    处理合并后的DataFrame,去除重复列并重命名列名
    Args:
        df: 需要处理的DataFrame
    Returns:
        处理后的DataFrame
    """
    # 去除重复列
    result = df.loc[:, ~df.columns.str.endswith('_y')]
    # 创建副本避免SettingWithCopyWarning
    result = result.copy()
    # 重命名列,将_x替换为空
    result = result.rename(columns=lambda x: x.replace('_x', ''))
    return result

# 处理各个结果
result_wlxz = process_merge_result(wlxz_result)
result_hxxz = process_merge_result(hxxz_result)
result_hjxz = process_merge_result(hjxz_result) 
result_trrz = process_merge_result(trrz_result)


In [18]:
# 过滤数据
# 过滤数据 土壤容重结果 TRRZPJZ不为空
filter_trrz_result = result_trrz[
    (~result_trrz['TRRZPJZ'].isna()) & # 过滤 NaN
    (result_trrz['TRRZPJZ'] != 'None') & # 过滤字符串 'None' 
    (result_trrz['TRRZPJZ'].notna()) & # 再次确认过滤 NaN
    (result_trrz['TRRZPJZ'] != '') # 过滤空字符串

]
print(filter_trrz_result.shape[0])
filter_trrz_result.sample(1)
# 删除YYPH重复项
filter_trrz_result = remove_duplicate(filter_trrz_result,'YPBH')
print(filter_trrz_result.shape[0])
filter_trrz_result.sample(1)
# 删除YDBH重复项
filter_trrz_result = remove_duplicate(filter_trrz_result,'YDBH')
print(filter_trrz_result.shape[0])
filter_trrz_result.sample(1)
# 查看处理前后的YDBH数量
print(f"原YDBH数量：{trrz_df['YDBH'].nunique()}")
print(f"去重后YDBH数量：{filter_trrz_result['YDBH'].nunique()}")


1684
1684
1684
原YDBH数量：1712
去重后YDBH数量：1684


In [19]:
# 过滤数据 土壤物理性状结果 YPLX为1或2，CH为1或空值
# filter_wlxz_result = result_wlxz[
#     (result_wlxz['YPLX'].isin(['1','2',])) & # YPLX为1或2
#     ((result_wlxz['CH'].isna()) | (result_wlxz['CH'] == '1')) # CH为空值或1
# ]
filter_wlxz_result = result_wlxz[
    ((result_wlxz['CH'].isna()) | (result_wlxz['CH'] == '1')) # CH为空值或1
]
print(filter_wlxz_result.shape[0])
filter_wlxz_result.sample(1)
# 删除YYPBH
filter_wlxz_result = remove_duplicate(filter_wlxz_result,'YYPBH')
print(filter_wlxz_result.shape[0])
filter_wlxz_result.sample(1)
# 删除YDBH
filter_wlxz_result = remove_duplicate(filter_wlxz_result,'YDBH')
print(filter_wlxz_result.shape[0])
filter_wlxz_result.sample(1)
# 查看处理前后的YDBH数量
print(f"原YDBH数量：{wlxz_df['YDBH'].nunique()}")
print(f"去重后YDBH数量：{filter_wlxz_result['YDBH'].nunique()}")



2045
1998
1802
原YDBH数量：1802
去重后YDBH数量：1802


In [20]:
# 过滤数据 土壤化学性状结果 
filter_hxxz_result = result_hxxz[
    ((result_hxxz['CH'].isna()) | (result_hxxz['CH'] == '1')) # CH为空值或1
]
print(filter_hxxz_result.shape[0])
filter_hxxz_result.sample(1)
# 删除YYPBH
filter_hxxz_result = remove_duplicate(filter_hxxz_result,'YYPBH')
print(filter_hxxz_result.shape[0])
filter_hxxz_result.sample(1)
# 删除YDBH
filter_hxxz_result = remove_duplicate(filter_hxxz_result,'YDBH')
print(filter_hxxz_result.shape[0])
filter_hxxz_result.sample(1)
# 查看处理前后的YDBH数量
print(f"原YDBH数量：{hxxz_df['YDBH'].nunique()}")
print(f"去重后YDBH数量：{filter_hxxz_result['YDBH'].nunique()}")

1844
1802
1802
原YDBH数量：1802
去重后YDBH数量：1802


In [21]:
# 过滤数据 土壤环境性状结果 
filter_hjxz_result = result_hjxz[
    ((result_hjxz['CH'].isna()) | (result_hjxz['CH'] == '1')) # CH为空值或1
]
print(filter_hjxz_result.shape[0])
filter_hjxz_result.sample(1)
# 删除YYPBH
filter_hjxz_result = remove_duplicate(filter_hjxz_result,'YYPBH')
print(filter_hjxz_result.shape[0])
filter_hjxz_result.sample(1)
# 删除YDBH
filter_hjxz_result = remove_duplicate(filter_hjxz_result,'YDBH')
print(filter_hjxz_result.shape[0])
filter_hjxz_result.sample(1)
# 查看处理前后的YDBH数量
print(f"原YDBH数量：{hjxz_df['YDBH'].nunique()}")
print(f"去重后YDBH数量：{filter_hjxz_result['YDBH'].nunique()}")

1844
1802
1802
原YDBH数量：1802
去重后YDBH数量：1802


In [22]:
# 处理后检查机械组成（filter_wlxz_result中TRZD列不为空且不为'/'的数据）和原表数据涉及的YDBH是否一致
jxzc_df = filter_wlxz_result[
    (filter_wlxz_result['TRZD'].notna()) & # 过滤NaN
    (filter_wlxz_result['TRZD'] != '/') # 过滤'/'
]
# 处理前机械组成YDBH数量
jxzc_df_before = wlxz_df[
    (wlxz_df['TRZD'].notna()) & # 过滤NaN
    (wlxz_df['TRZD'] != '/') # 过滤'/'
]
# 查看处理前后的YDBH数量及是否检测(SFJCJXZC列值为'1')的数量
print(f"处理前机械组成YDBH数量：{jxzc_df_before['YDBH'].nunique()}")
print(f"处理后机械组成YDBH数量：{jxzc_df['YDBH'].nunique()}")
print(f"是否检测机械组成YDBH数量：{jxzc_df[jxzc_df['SFJCJXZC'] == '1']['YDBH'].nunique()}")


处理前机械组成YDBH数量：919
处理后机械组成YDBH数量：867
是否检测机械组成YDBH数量：864


In [23]:
# 处理后水团组成（filter_wlxz_result中SWXDTJT7列不为空且不为'/'的数据）和原表数据涉及的YDBH是否一致
stzc_df = filter_wlxz_result[
    (filter_wlxz_result['SWXDTJT7'].notna()) & # 过滤NaN
    (filter_wlxz_result['SWXDTJT7'] != '/') # 过滤'/'
]
# 处理前机械组成YDBH数量
stzc_df_before = wlxz_df[
    (wlxz_df['SWXDTJT7'].notna()) & # 过滤NaN
    (wlxz_df['SWXDTJT7'] != '/') # 过滤'/'
]
# 查看处理前后的YDBH数量及是否采集水团样品(SFCJSWXDTJTYP列值为'1')
print(f"处理前水团组成YDBH数量：{stzc_df_before['YDBH'].nunique()}")
print(f"处理后水团组成YDBH数量：{stzc_df['YDBH'].nunique()}")
print(f"是否采集水团样品YDBH数量：{stzc_df[stzc_df['SFCJSWXDTJTYP'] == '1']['YDBH'].nunique()}")


处理前水团组成YDBH数量：196
处理后水团组成YDBH数量：93
是否采集水团样品YDBH数量：93


In [24]:
# 连接所有表 filter_wlxz_result filter_hxxz_result filter_hjxz_result filter_trrz_result,ldtj_df pmxx_df 基于YDBH
result_all = pd.merge(filter_wlxz_result,filter_hxxz_result,on='YDBH',how='left')
result_all = process_merge_result(result_all)
result_all = pd.merge(result_all,filter_hjxz_result,on='YDBH',how='left')
result_all = process_merge_result(result_all)
result_all = pd.merge(result_all,filter_trrz_result,on='YDBH',how='left')
result_all = process_merge_result(result_all)
result_all = pd.merge(result_all,ldtj_df,on='YDBH',how='left')
result_all = process_merge_result(result_all)
result_all = pd.merge(result_all,pmxx_df,on='YDBH',how='left')
result_all = process_merge_result(result_all)

In [25]:
result_all

,YDBH,YDLB,CYLX,BSJD,BSWD,JD,WD,DWJD,DWWD,DWGC,...,MZ,MZQT,TDLYLX,GZCHD,FSCS,YXTCHD,FSXTL,FSXYL,FSXTS,FSXTZ
0,5203260101000001,0,1,107.87895967147765000000,28.70858850924832200000,107.87895967147765000000,28.70858850924832200000,107.87884915613530000000,28.70840749416682000000,612.89843750000000000000,...,LG,None,0103,13.00000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
1,5203260101000006,0,1,107.83319762443762000000,28.19526752962602600000,107.83319762443762000000,28.19526752962602600000,107.83365779068251000000,28.19492551926570200000,779.96868106388300000000,...,LG,None,0101,19.00000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
2,5203260101000011,0,1,108.04757353618687000000,28.99026463645311200000,108.04757353618687000000,28.99026463645311200000,108.04752500000001000000,28.99035333333334000000,681.80000000000000000000,...,LG,None,0103,16.70000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
3,5203260101000047,0,1,107.81973285355339000000,28.69885644896205800000,107.81973285355339000000,28.69885644896205800000,107.81957833333333000000,28.69888166666666000000,1127.70000000000000000000,...,LG,None,0101,20.80000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
4,5203260101000049,0,1,108.05213277794779000000,28.76358157101752500000,108.05222023000000000000,28.76235333000000000000,108.05222023000000000000,28.76235333000000000000,398.25177001953125000000,...,LG,None,0103,13.00000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1797,5203260305001764,0,1,108.03692156360263000000,28.57268396345814700000,108.03370800000000000000,28.57315200000000000000,108.03374190000000000000,28.57331150000000000000,1219.64978027343750000000,...,LI,None,0305,None,NaN,NaN,NaN,NaN,NaN,NaN
1798,5203260305100007,1,1,107.98891591484566000000,28.47613385407711600000,107.98891591484566000000,28.47613385407711600000,107.98962333333333000000,28.47587166666666300000,1201.60000000000000000000,...,LG,None,0305,None,2.0,50.0,石灰（岩）土,黄色石灰土,壤质黄色石灰土,腐中层壤质黄色石灰土
1799,5203260305100014,1,1,108.02390809687263000000,28.66590182540001300000,108.02390809687263000000,28.66590182540001300000,108.02445856000000000000,28.66538636000000000000,1032.52978515625000000000,...,LG,None,0305,None,3.0,48.0,石灰（岩）土,黄色石灰土,壤质黄色石灰土,腐中层壤质黄色石灰土
1800,5203260305100024,1,1,107.94606518307556000000,28.81575375642652000000,107.94606518307556000000,28.81575375642652000000,107.94693239000000000000,28.81618876000000000000,757.44677734375000000000,...,LG,None,0301,None,5.0,120.0,石灰(岩)土,黑色石灰土,壤质黑色石灰土,腐厚层壤质黑色石灰土


In [26]:
# 导出为xlsx,总表
save_path = r"E:\soil_property_result\wcx\table"
os.makedirs(save_path,exist_ok=True)
result_all.to_excel(os.path.join(save_path,f'result_all_{time.strftime("%Y%m%d_%H%M%S")}.xlsx'),index=False)

In [28]:
# 导出为xlsx,用于分析
# use_col YDBH DWJD DWWD DWGC TRZD PH CEC OM TN TP TK TSE AP SK AK HG AS2 PB CD CR TRRZPJZ GZCHD_LDTJ GZCHD YXTCHD
ana_df = result_all[['YDBH','DWJD','DWWD','DWGC','TRZD','PH','CEC','OM','TN','TP','TK','TSE','AP','SK','AK',
                     'HG','AS2','PB','CD','CR','TRRZPJZ','GZCHD','YXTCHD','FSXTL', 'FSXYL', 'FSXTS', 'FSXTZ']]
# 重命名列
ana_df = ana_df.copy()
ana_df.rename(columns={'TRRZPJZ':'TRRZ'},inplace=True)
ana_df.to_excel(os.path.join(save_path,f'result_ana_df_{time.strftime("%Y%m%d_%H%M%S")}.xlsx'),index=False)